# attention中的query_key_value基本概念


## 相关代码
### repeat_kv

In [3]:
import torch
def repeat_kv(x: torch.Tensor, n_rep: int) -> torch.Tensor:
    '''这个函数通常被用于在某个特定的维度上复制张量，这样可以创建出具有更高维度的数据。
    x : 输入的张量
    n_rep : 复制次数
    '''
    # bs : batch size
    # slen : sequence length
    # n_kv_heads : number of key/value heads
    # head_dim : head dimension
    bs, slen, n_kv_heads, head_dim = x.shape
    
    # 如果n_rep等于1，那就没有必要复制张量，函数直接返回输入张量x。
    if n_rep == 1:
        return x 
    
    # x[:, :, :, None, :]  添加新维度
    # 通过在第四个维度位置插入一个新的维度（使用None或np.newaxis）来增加输入张量x的维度。
    
    # .expand(bs, slen, n_kv_heads, n_rep, head_dim) 张量扩展
    # 使用expand方法来扩展张量的新维度，从而复制该维度。n_rep参数控制了新维度的大小，即复制的次数。
    
    # .reshape(bs, slen, n_kv_heads * n_rep, head_dim) 重塑张量
    # 使用reshape方法将扩展后的张量重塑为最终的形状，这里通过乘以n_rep将 _kv_heads和新添加的维度合并为一个维度，从而得到最终的形状。
    return (
        x[:, :, :, None, :]
        .expand(bs, slen, n_kv_heads, n_rep, head_dim)
        .reshape(bs, slen, n_kv_heads * n_rep, head_dim)
    )

In [4]:
# 1. 准备数据

# 输入数据
x = torch.tensor([
    [[1, 2], [3, 4]],
    [[5, 6], [7, 8]]
])

# 复制次数
n_rep = 3

print(x.shape)

# 2. 调用上面定义的函数

# 为 x 增加一个维度，匹配四个维度: bs, slen, n_kv_heads, head_dim
x = x.unsqueeze(-1) # x shape : (2, 2, 2, 1)

# 调用函数
output_tensor = repeat_kv(x, n_rep)

print(output_tensor.shape)

torch.Size([2, 2, 2])
torch.Size([2, 2, 6, 1])


## 计算Query/Key/Value向量原理

在自注意力机制中，输入是一个序列的向量表示，每个向量都要被转换为三个不同的向量：Query（查询）向量、Key（键）向量和Value（值）向量。这三个向量是通过线性变换得到的，即通过与权重矩阵相乘来计算。  


对于 Query 向量的计算，公式如下：  
$Query$ = $X \cdot W_Q$   
其中: 
- $X$是输入向量，其形状为: (batch_size, seq_len, dim)  
- $W_Q$ 是 Query向量的权重矩阵，其形状为 (dim, n_head $\cdot$ head_dim)  
- $Query$是计算得到Query向量，其形状为 (batch_size, seq_len, n_head $\cdot$ head_dim)   

$Key$ 和 $Value$ 向量的计算原理与上一致。  

$Key = X \cdot W_k$

$Value = X \cdot W_v$  